# Загрузка часовых котировок за 2010 год (3-месячными кусками)

В этом ноутбуке:
1. Запрашиваем данные с OANDA (демо-среда) по инструменту EUR_USD.
2. Делим период 2010-01-01 ... 2011-01-01 на интервалы по **3 месяца**.
3. Склеиваем все куски в один DataFrame.
4. Удаляем дубликаты на границах интервалов.
5. Сортируем и сохраняем итоговый CSV.

In [4]:
# Шаг 1. Импорт необходимых библиотек
import requests
import pandas as pd
import datetime as dt
import yaml
import os

# Загружаем конфигурацию из YAML файла
config_path = os.path.join(os.path.dirname(os.getcwd()), '..', '04_configs', 'oanda_config.yml')

with open(config_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Извлекаем настройки из конфигурации
API_TOKEN = config['api_token']
BASE_URL_DEMO = config['api']['base_url_demo']
DEFAULT_INSTRUMENT = config['trading']['default_instrument']
DEFAULT_GRANULARITY = config['data_extraction']['default_granularity']
DEFAULT_PRICE_TYPE = config['data_extraction']['default_price_type']
DEFAULT_COUNT = config['data_extraction']['default_count']

**Позже выяснилось, что данные можно было скачать по ссылке с Dukascopy:** https://www.dukascopy.com/swiss/english/marketwatch/historical/

## Настройка ключа доступа, базовых переменных
- `API_TOKEN`: твой токен от OANDA
- `base_url`: для демо (practice) или боевого (live)
- `instrument`: "EUR_USD"
- `start_all` и `end_all`: границы периода
- `delta`: размер шага (по 3 месяца)

In [ ]:
# Настройки API берутся из файла конфигурации
base_url = BASE_URL_DEMO
instrument = DEFAULT_INSTRUMENT

# Настройки периода из конфигурации
start_all = dt.datetime(config['data_period']['start_year'], 1, 1)
end_all   = dt.datetime(config['data_period']['end_year'], 1, 1)

# Сколько месяцев запрашиваем за один раз (из конфигурации)
delta = pd.DateOffset(months=config['data_period']['chunk_months'])


## Функция для выгрузки данных за один промежуток
**fetch_oanda_data**:
- Принимает start_dt, end_dt (datetime)
- Делает GET-запрос к OANDA
- Возвращает DataFrame (time, open, high, low, close, volume)

In [ ]:
def fetch_oanda_data(start_dt, end_dt, granularity=DEFAULT_GRANULARITY, price=DEFAULT_PRICE_TYPE, count=DEFAULT_COUNT):
    """
    Загружает часовые свечи OANDA между start_dt и end_dt (UTC).
    Возвращает DataFrame c колонками: time, open, high, low, close, volume.
    """
    url = f"{base_url}/instruments/{instrument}/candles"
    
    from_str = start_dt.isoformat(timespec='seconds') + "Z"
    to_str   = end_dt.isoformat(timespec='seconds') + "Z"
    
    params = {
        "from": from_str, 
        "to": to_str,
        "granularity": granularity,
        "price": price,
        # "count": count  # Если раскомментировать, то вернет count свечей
    }
    
    headers = {
        "Authorization": f"Bearer {API_TOKEN}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers, params=params)
    data_json = response.json()
    
    # Извлекаем массив свечей
    candles = data_json.get("candles", [])
    
    records = []
    for c in candles:
        time_utc = c["time"]
        volume   = c["volume"]
        mid      = c["mid"]  # {"o": ..., "h": ..., "l": ..., "c": ...}
        
        o = float(mid["o"])
        h = float(mid["h"])
        l = float(mid["l"])
        cl= float(mid["c"])
        
        records.append([time_utc, o, h, l, cl, volume])
    
    df_part = pd.DataFrame(records, columns=["time", "open", "high", "low", "close", "volume"])
    df_part["time"] = pd.to_datetime(df_part["time"])
    return df_part


## Цикл по 3-месячным интервалам
- Будем хранить куски в списке `dfs`.
- В конце объединим их в `df_full`.

In [ ]:
dfs = []
current_start = start_all

while current_start < end_all:
    current_end = current_start + delta
    # если вышли за верхнюю границу, обрезаем
    if current_end > end_all:
        current_end = end_all
    
    print(f"Запрашиваем данные с {current_start.date()} по {current_end.date()}...")
    
    df_chunk = fetch_oanda_data(current_start, current_end)
    
    dfs.append(df_chunk)
    
    # шаг к следующему куску
    current_start = current_end

# Объединяем всё
df_full = pd.concat(dfs, ignore_index=True)

# Удаляем возможные дубликаты (на стыках диапазонов)
df_full.drop_duplicates(subset=["time"], inplace=True)

# Сортируем
df_full.sort_values("time", inplace=True)
df_full.reset_index(drop=True, inplace=True)

print(f"Всего строк после объединения: {len(df_full)}")

Запрашиваем данные с 2010-01-01 по 2010-04-01...
Запрашиваем данные с 2010-04-01 по 2010-07-01...
Запрашиваем данные с 2010-07-01 по 2010-10-01...
Запрашиваем данные с 2010-10-01 по 2011-01-01...
Запрашиваем данные с 2011-01-01 по 2011-04-01...
Запрашиваем данные с 2011-04-01 по 2011-07-01...
Запрашиваем данные с 2011-07-01 по 2011-10-01...
Запрашиваем данные с 2011-10-01 по 2012-01-01...
Запрашиваем данные с 2012-01-01 по 2012-04-01...
Запрашиваем данные с 2012-04-01 по 2012-07-01...
Запрашиваем данные с 2012-07-01 по 2012-10-01...
Запрашиваем данные с 2012-10-01 по 2013-01-01...
Запрашиваем данные с 2013-01-01 по 2013-04-01...
Запрашиваем данные с 2013-04-01 по 2013-07-01...
Запрашиваем данные с 2013-07-01 по 2013-10-01...
Запрашиваем данные с 2013-10-01 по 2014-01-01...
Запрашиваем данные с 2014-01-01 по 2014-04-01...
Запрашиваем данные с 2014-04-01 по 2014-07-01...
Запрашиваем данные с 2014-07-01 по 2014-10-01...
Запрашиваем данные с 2014-10-01 по 2015-01-01...
Запрашиваем данные с

In [17]:
df_full.head()

,time,open,high,low,close,volume
0,2010-01-03 17:00:00+00:00,1.43120,1.43172,1.43120,1.43172,3
1,2010-01-03 18:00:00+00:00,1.43172,1.43425,1.43105,1.43157,137
2,2010-01-03 19:00:00+00:00,1.43154,1.43212,1.43081,1.43106,299
3,2010-01-03 20:00:00+00:00,1.43103,1.43201,1.43076,1.43080,261
4,2010-01-03 21:00:00+00:00,1.43078,1.43114,1.42926,1.43036,327


In [18]:
df_full.tail()

,time,open,high,low,close,volume
88625,2023-12-29 17:00:00+00:00,1.10607,1.10669,1.10529,1.10643,4230
88626,2023-12-29 18:00:00+00:00,1.10641,1.10641,1.10493,1.10532,5238
88627,2023-12-29 19:00:00+00:00,1.10530,1.10540,1.10382,1.10410,2778
88628,2023-12-29 20:00:00+00:00,1.10411,1.10510,1.10384,1.10411,2265
88629,2023-12-29 21:00:00+00:00,1.10410,1.10413,1.10342,1.10374,1250


## Сохраняем результат
Сохраняем в CSV, выводим конец таблицы для проверки.

In [ ]:
df_full.to_csv("EURUSD_2010-2024_H1_OANDA.csv", index=False)
print("Сохранён файл: EURUSD_2010-2024_H1_OANDA.csv")


Сохранён файл: EURUSD_2010_H1.csv
